## Exercise 11: XPS with GW

During the lecture you have learned a graphical way to get the HOMO state quasi-particle energy of benzene from the G<sub>0</sub>W<sub>0</sub> calculation. The solutions are the intersections between a special strainght line and the real part of the correlation part of the self energy Σ<sup>c</sup>. It looks like Figure 11. A in [this review](https://www.frontiersin.org/articles/10.3389/fchem.2019.00377/full). The x-coordinates of the intersection points corresponding to the quasi-particle energies. 

In principle, the best result should be only one intersection point with relative flat slope of Σ<sup>c</sup> at that point. A sharp slope at the intersection point corresponds to a week peak in the XPS, whereas a flat slope corresponds to a sharp peak in the XPS. Two or more intersection points with flat slopes might be a sign of G<sub>0</sub>W<sub>0</sub> is insufficient for this calculation. 

Anyhow, there is also a self-consistent way to get the quasi-particle energy too. More specifically, we will use analytic continuation techinque to obtain an analytical expression of the Σ<sup>c</sup> and keep iterating the quasi-particle energy using this analytical expression to get the final converged Σ<sup>c</sup> and quasi-particle energy. 

In this exercise, you will need to complete the iteration code to get the converged HOMO state quasi-particle energy of the benzene, and compare this result with the graphical solution you get from the lecture.

In [75]:
!pip install numpy
!pip install matplotlib

In [76]:
import numpy as np
import matplotlib.pyplot as plt

You could use the following function `get_real_self_energy` as a block box. It is a pre-calculated analytical expression of the Σ<sup>c</sup> of benzene. The input of this function is the quasi-particle enery minus the chemical potential energy, and the output is the Σ<sup>c</sup>. 

The result is converged meaning that the difference between the input quasi-particle energy and the quasi-particle energy calculated from the output Σ<sup>c</sup> are smaller than the given threshold. 

In [77]:
# This is a pre-calculated analytical expression of the Σ^c of the benzene.
# The input of this function is the quasi-particle energy - chemical potential energy, and it returns the Σ<sup>c</sup>
def get_real_self_energy(qp_energy):
    n_freq = 60
    n_par = 16
    n_step = n_freq/(n_par-1)
    i_dat = int(0)
    
    omega = np.array([1.9754721211817734E-004, 
1.0420358813272906E-003,
2.5661289834708343E-003,
4.7784341264780024E-003,
7.6910655734194590E-003,
1.1320049160300182E-002,
1.5685535746168384E-002,
2.0812046177788337E-002,
2.6728769554759748E-002,
3.3469923642080790E-002,
4.1075185768400330E-002,
4.9590203896067518E-002,
5.9067199529567821E-002,
6.9565676597037190E-002,
8.1153253448940504E-002,
9.3906638788311195E-002,
0.10791277684402006    , 
0.12327019263883966    , 
0.14009057506787503    , 
0.15850064405408279    , 
0.17864435876090740    , 
0.20068553734092659    , 
0.22481097580763218    , 
0.25123417543113491    , 
0.28019981604633737    , 
0.31198914880653561    , 
0.34692652891375569    , 
0.38538737040915960    , 
0.42780788633142453    , 
0.47469708559525736    , 
0.52665164288191646    , 
0.58437445401912003    , 
0.64869795742029379    , 
0.72061367224571293    , 
0.80130992041336968    , 
0.89222042871954221    , 
0.99508754957793077    , 
1.1120453487730144     ,
1.2457300277463319     ,
1.3994284607363001     ,
1.5772806570722593     ,
1.7845597384326028     ,
2.0280652982546781     ,
2.3166858208213519     ,
2.6622185952535462     ,
3.0805912193932365     ,
3.5937262775166845     ,
4.2324674606395574     ,
5.0413182515634887     ,
6.0863997404566383     ,
7.4693926007552065     ,
9.3532176813384602     ,
12.012273942905843     ,
15.938250630748730     ,
22.084709744614798     ,
32.505248799855131     ,
52.318394139769225     ,
97.423006251954206     ,
239.91496308319356     ,
1265.5202638366984])
    
    par = np.array([ complex(  -9.5031302643326064E-003,  -1.9662929629954775E-005), 
complex(  -10.407175262526630     ,  0.82747552149990167     ),
complex(   10.273752133547323     , -0.86943582891677063     ),
complex(  0.14220459081555178     ,   1.4888263128107378E-002),
complex( -0.90497522086867310     , -0.69709676173252100     ),
complex(  -2.6144766199669554     , -0.35492120583872061     ),
complex(  0.18604167241926894     , -0.58822095483356107     ),
complex(  0.57038791214693518     ,  0.38822662740643443     ),
complex(  0.32538298996151099     ,  -1.2922701827738436     ),
complex( -0.56647797834979707     ,  0.15755627248949919     ),
complex(  -3.2601254208423788E-002,  -6.0000456904019152E-002),
complex(  0.23901709401688750     , -0.24269112029419374     ),
complex(  -6.2685338644493695E-003,   4.4742789294792674E-002),
complex( -0.30172684243028941     ,   6.0343292442250930E-002),
complex(   1.2842882757477849E-002,  -3.3355298637269368E-002),
complex(  -1.7180313672644693E-003,  -1.1523099464934327E-002)])
    
    xdata=np.array([])
    for i_par in np.arange(n_par-1):
        xdata = np.append(xdata,complex(0.0,omega[i_dat]))
        i_dat = int(i_dat + n_step)
    
    xdata = np.append(xdata,complex(0.0,omega[n_freq-1]))

    gtmp = complex(1.0,0.0)
    for i_par in np.arange(n_par, 1, -1):
        gtmp = 1.0 + par[i_par-1]*(complex(qp_energy,0.0)-xdata[i_par-2])/gtmp
        
    self_energy = par[0]/gtmp
    
    return self_energy
    

In [78]:
# setup the constants

# Convergence threshold
qp_energy_thr = 10**(-5) 

# Adjust_coefficient, qp_adj * energy_diff = quasi-particle energy adjustment 
qp_adj = 0.1

# Chemical potential of the benzene system, this value needs to be subtracted from the quasi-particle energy 
# before putting into the get_real_self_energy function.
# This is a pre-calculated value too. The unit is eV.
chemical_potential = -3.8526

# The exchange correlation potential v_xc. The unit is eV.
v_xc = -13.5051

# The exchange part of the self-energy sigma_x. The unit is eV.
sigma_x = -16.0698

# The Kohn-Sham energy e_ks. The unit is eV.
e_ks = -6.3048

The quasi-particle energy is calculated based on: 

**qp_energy = e_ks + sigma_x - v_xc + Σ^c**


Please finish the remaining part of the code. 

***Hints***: The iteration loop should be like this;
1. Using *e_ks* as initial qp_energy, subtract chemical potential, i.e. *qp_energy* = *qp_energy* - *chemical_potential*, call `get_real_self_energy(qp_energy)`, and obtain the first exchange self energy Σ^c.
2. Put previous Σ^c into above *qp_energy* expression and get next *qp_energy*, subtract chemical_potential and then call `get_real_self_energy(qp_energy)` again. 
3. Keep running this loop until the qp_energy is converged, i.e. the difference is smaller than *qp_energy_thr*.
4. This final qp_energy is the HOMO state quasi-particle energy of benzene.

Please compare this qp_energy with the one you get from the graphical solution, they should be the same or very close.